## Cooling Tank Model

This notebook demonstrates the optimization of the tank model described in `tank_optimization_model.py` using the `ProblemInterface` class in the `fmdtools.search` module.

Note that this problem/notebook was adapted from the identical problem here: https://github.com/DesignEngrLab/resil_opt_examples/tree/main/Cooling%20Tank%20Problem, which shows more comparisions of these architectures. 

In [1]:
import numpy as np
import time
import pandas as pd

import fmdtools.sim.propagate as propagate
import fmdtools.analyze as an
import fmdtools.sim.search as search
from tank_optimization_model import Tank
from fmdtools.define import SampleApproach
import matplotlib.pyplot as plt

ImportError: cannot import name 'SampleApproach' from 'fmdtools.define' (c:\users\dhulse\documents\github\fmdtools\fmdtools\define\__init__.py)

# Simulation

### Verifying the nominal state:

In the nominal state, no change in system state should occurs.

In [2]:
mdl = Tank()
resgraph, mdlhist = propagate.nominal(mdl, desired_result='fxngraph')

In [3]:
an.graph.show(resgraph, gtype='fxngraph')

In [4]:
an.plot.mdlhistvals(mdlhist)

### What happens under component faults?


Here we model a leak of the tank. As shown, the coolant leaks until there is no more coolant left in the tank. While this results in a warning signal, the default contingency management policy is to take no actions to alleviate the condition.

In [5]:
resgraph, mdlhist = propagate.one_fault(mdl,'Store_Coolant','Leak', time=3, gtype='fxngraph', desired_result='fxngraph')

fig, ax = an.plot.mdlhistvals(mdlhist, fault='Leak', time=3, fxnflowvals={'Store_Coolant':['level', 'net_flow'], 'Tank_Sig':['indicator'], 'Valve1_Sig':['action']}, legend=False) #,
#fig.axes[3].remove()
fig.set_figheight(6)
fig.set_figwidth(6)
fig.subplots_adjust(top = 0.9, wspace=0.1, hspace=0.3)

In [6]:
#add graph view to figure
import networkx as nx
graph_fig, graph_ax = an.graph.show(resgraph,faultscen='Leak', time=3, scale=0.8, gtype='fxngraph', pos=nx.shell_layout(resgraph))
graph_ax.figure = fig
fig.axes.append(graph_ax)
fig.add_axes(graph_ax)
graph_ax.set_position([0.6,0.10,0.5,0.5])
graph_ax.margins(0.3)

In [7]:
fig

### Full set of modes

The tank leak mode will not be the only mode considered, but also leak and blockage faults in the Imput/Output blocks.

In [8]:
app_joint_faults = SampleApproach(mdl)
endclasses, mdlhists = propagate.approach(mdl, app_joint_faults)
fmea_tab = an.tabulate.simplefmea(endclasses)
fmea_tab

In [9]:
print(fmea_tab.to_latex(float_format="%.2g"))

### Defining Optimization Problem

We can define an optimization problem around this model using the `ProblemInterface` class.

In this case, we consider the joint minimization of design cost (defined in a function) and resilience cost (expected cost over the above list of scenarios). This is over two sets of variables

- capacity and turnup (design variables that effect both design and resilience costs), and
- the resilience policy (variables that only effect resilience cost)

In [10]:
mdl= Tank()
prob = search.ProblemInterface("res_problem", mdl, staged=True)

To model design cost (which does not involve simulation), we attach an external callable using the "external" option in `.add_simulation`. Note that this callable is defined in terms of its objective only.

In [11]:
def x_to_descost(xdes):
    pen = 0 #determining upper-level penalty
    if xdes[0]<10: pen+=1e5*(10-xdes[0])**2
    if xdes[0]>100: pen+=1e5*(100-xdes[0])**2
    if xdes[1]<0: pen+=1e5*(xdes[1])**2
    if xdes[1]>1: pen+=1e5*(1-xdes[1])**2
    return (xdes[0]-10)*1000 + (xdes[0]-10)**2*1000   + xdes[1]**2*10000 + pen
prob.add_simulation("des_cost", "external", x_to_descost)
prob.add_objectives("des_cost", cd="cd")
prob.add_variables("des_cost",'capacity', 'turnup')

In [12]:
prob

Since the design variables affect both design and resilience costs, we can use the `upstream_sims` option to translate "des_cost" variables into "res_sim" simulation parameters. 

In [13]:
app = SampleApproach(mdl)
prob.add_simulation("res_sim", "multi", app.scenlist, upstream_sims = {"des_cost":{'params':{"capacity":"capacity", "turnup":"turnup"}}})

res_vars_i = {param:1 for param,v in mdl.params.items() if param not in ['capacity','turnup']}
res_vars = [(var, None) for var in res_vars_i.keys()]
prob.add_variables("res_sim", *res_vars, vartype="param")
prob.add_objectives("res_sim", cost="expected cost", objtype="endclass")

Finally, to get a callable for both objectives, we can use `add_combined_objective`, which will perform an opteration (default is sum) to combine the objectives:

In [14]:
prob.add_combined_objective('tot_cost', 'cd', 'cost')

In [15]:
prob

Note that because the simulations are linked, changes to xdes in the `cd` function in turn change the lower-level `cost` function by default.

In [16]:
xdes = [1,2]
prob.cd(xdes)

In [17]:
prob.cost([*res_vars_i.values()])

In [18]:
xdes = [1,3]
prob.cd(xdes)

In [19]:
prob.cost([*res_vars_i.values()])

These costs can be treated as independent by clearing the problem prior to the lower-level cost evalutation.

In [20]:
prob.clear()
prob.update_sim_vars("res_sim", newparams={'capacity':1, 'turnup':2})
prob.cost([*res_vars_i.values()])

### Optimization

This problem is optimized here using four different architectures, following the example in:

Hulse, D., & Hoyle, C. (2022). Understanding Resilience Optimization Architectures: Alignment and Coupling in Multilevel Decomposition Strategies. Journal of Mechanical Design, 144(11), 111704.

The implementation of these architectures is provided in `tank_opt` using the `ProblemInterface` class. Here we show some results for comparing the architectures, which as shown are consistent with the reference.

In [21]:
import importlib
import tank_opt
import multiprocessing as mp
importlib.reload(tank_opt)
pool = mp.Pool(5)

alternating optimization structure

In [22]:
result_alt, llargs_alt, fhist_alt, thist_alt, xdhist_alt = tank_opt.alternating_opt(pool=pool)

In [23]:
result_alt.fun

In [24]:
result_alt_nocr, llargs_alt_nocr, fhist_alt_nocr, thist_alt_nocr, xdhist_alt_nocr = tank_opt.alternating_opt(option="without_cr",pool=pool)

In [25]:
result_bi, llargs_bi, bestf_bi, bestxdes_bi = tank_opt.bilevel_opt(pool=pool)

In [26]:
fig = plt.figure()
fhist_alt_plot = [min(fhist_alt[:f+1]) for f,_ in enumerate(fhist_alt)]
plt.plot(thist_alt, fhist_alt_plot, label="Alternating (with $C_R$)")

fhist_alt_nocr_plot = [min(fhist_alt_nocr[:f+1]) for f,_ in enumerate(fhist_alt_nocr)]
plt.plot(thist_alt_nocr, fhist_alt_nocr_plot, label="Alternating  (no $C_R$)")

thist_bilevel_plot = [0]+llargs_bi['thist']
fhist_bilevel_plot = [fhist_alt[0]]+llargs_bi['fhist']
fhist_bilevel_plot = [min(fhist_bilevel_plot[:f+1]) for f,_ in enumerate(fhist_bilevel_plot)]
plt.plot(thist_bilevel_plot, fhist_bilevel_plot, label="Bilevel")
#plt.plot(t, [fhist_alt[0]]+llargs_bi['fhist'], label="Bilevel")
plt.title("Optimization of Tank Problem")
plt.grid()
plt.ylabel("$C_D+C_R$")
#plt.yscale("log")
plt.xlabel("Computational Time (s)")
plt.legend()

In [27]:
fig.savefig('tank_optimization.pdf', format="pdf", bbox_inches = 'tight', pad_inches = 0.0)

In [28]:
tab = pd.DataFrame(columns = ["$x_t$", "$x_l$", "$f^*$", "time"])
tab.loc['Bilevel'] = list(result_bi['x']) +  [result_bi['fun'], llargs_bi['thist'][-1]]
tab.loc['Alt. (no $C_R$)'] = list(result_alt_nocr['x']) + [llargs_alt_nocr['ll_opt'], thist_alt_nocr[-1]]
tab.loc['Alt. (with $C_R$)'] = list(result_alt['x']) + [result_alt['fun'], thist_alt[-1]]
tab.loc['Seq. (with $C_R$)'] = list(xdhist_alt[1]) + [fhist_alt[1], thist_alt[1]]
tab.loc['Seq. (no $C_R$)'] = list(xdhist_alt_nocr[1]) + [fhist_alt_nocr[1], thist_alt_nocr[1]]

In [29]:
tab

Note that these are the previously-recorded results:

|               |    𝑥_t      |   𝑥_l   |       𝑓     |    time     |
|--------------------|----------:|---------:|--------------:|------------|
|            Bilevel | 18.000015 | 0.580982 | 285708.991759 | 619.369699 |
|   Alt. (no  𝐶 𝑅CR) | 10.000000 | 0.000000 | 893333.333333 | 168.275383 |
| Alt. (with  𝐶 𝑅CR) | 20.000000 | 0.000000 | 452666.666823 | 306.776320 |
| Seq. (with  𝐶 𝑅CR) | 22.000000 | 0.000000 | 466333.333389 |  61.702104 |
|   Seq. (no  𝐶 𝑅CR) | 10.000000 | 0.000000 | 893333.333333 |  55.957946 |

As shown, the results are consistent (not exactly, since the lower-level EA is a stochastic search), and this problem gives better results from a bilevel search as expected. However, the optimization is somewhat slower, so it may (still) be helpful to implement interfaces manually to avoid some model instantiation overhead.